In [4]:
import requests
import time
import pandas as pd
import re

In [3]:
### Extraction of Reactome pathway names (2022-07-13)

In [23]:
standard_name = []
brief_description = []
gene = []
with open(r'../data/Reactome_pathway/c2.cp.reactome.v7.5.1.symbols.gmt') as f:
    for row in f:
        col = row.strip().split('\t')
        sn = col[0]
        url = col[1]
        r = requests.get(url, timeout = 30)
        bd = re.search(r'(.*)(Brief description</th>\n    <td>)(.*)(</td>)', r.text).group(3)
        for i in col[2:]:
            standard_name.append(sn)
            brief_description.append(bd)
            gene.append(i)

In [27]:
reactome_pathway = pd.DataFrame({'Standard_name': standard_name,
                                 'Brief_description': brief_description,
                                 'Gene': gene})

In [30]:
reactome_pathway.head()

,Standard_name,Brief_description,Gene
0,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,JAK2
1,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,TYK2
2,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,CBL
3,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,STAT1
4,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,IL6ST


In [33]:
reactome_pathway.to_csv(path_or_buf='../data/Reactome_pathway/reactome_pathways_from_MsigDB.txt',
                        index = False, 
                        sep = '\t')

In [57]:
### Extraction of pathway categories information (2022-12-16)

In [37]:
reactome_pathways = pd.read_csv(r'../data/Reactome_pathway/UniProt2Reactome_All_Levels.txt',
                                sep="\t",
                                header=None)

In [40]:
reactome_pathways = reactome_pathways[reactome_pathways[5] == "Homo sapiens"].iloc[:, [1, 3]].drop_duplicates()

In [56]:
pathway = []
category = []

for l in reactome_pathways.index:
    pathway.append(reactome_pathways.loc[l, 3])
    
    id = reactome_pathways.loc[l, 1]
    url = "https://reactome.org/content/detail/" + id
    r = requests.get(url, timeout = 30)
    
    if bool(re.search(r'title="goto Reactome Pathway Browser"', r.text)):
        ca = re.search(r'(.*)(title="goto Reactome Pathway Browser" >)(.*)( \(Homo sapiens\))', r.text).group(3)
    else:
        ca = re.search(r'(.*)(<i class="sprite-resize sprite sprite-Pathway" style="vertical-align: middle"></i>\n\s+\n\s+)(.*)( \(Homo sapiens\))', 
                             r.text).group(3)
        
    category.append(ca)

In [61]:
pathway_category = pd.DataFrame({'pathway': pathway, 'category': category})

In [63]:
pathway_category.to_csv(path_or_buf='../data/Reactome_pathway/reactome_pathway_category.txt',
                        index = False, sep = '\t')